## CSE202 - Disign and Analysis of Algorithms - Week 5 - Randomized Algorithms 1: Principle & First Examples

日期：2022-10-21

姓名：Yubo Cai

课程：CSE202 - Disign and Analysis of Algorithms 高级算法设计与分析

<img src="https://raw.githubusercontent.com/adimajo/CSE204-2021/master/data/logo.jpg" style="float: left; width: 15%" />

In [1]:
from sympy.matrices import randMatrix
import numpy as np
import matplotlib.pyplot as plt
import timeit
import math
import sympy as sym

### 1. Two Flavours of Randomized Algorithms

`Las Vegas:` algorithms always give the correct answer, but may take a long time to do so. 

`Monte Carlo:` algorithms may give the wrong answer, but with bounded probability.

### 2. Min-Cut in a Graph 最小割问题

#### 2.1. Global Minimum Cut 

Let $G = (V,E)$ be an **undirected multi-graph** (can have multiple edges between the same two vertices)

The `cut of a partition (S, S')` of $V$ is the set of edges that connect a vertex in $S$ to a vertex in $S'$.

`Global min cut.` Given G, find a cut $(S, S')$ of minimum cardinality.

**Applications**. Partitioning items in a database, network realiability, network design, etc.

#### 2.2 Karger's Algorithm

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e7/Single_run_of_Karger%E2%80%99s_Mincut_algorithm.svg" style="float:; width: 100%" />

[具体过程](https://en.wikipedia.org/wiki/Karger%27s_algorithm) 

`算法描述:` 对于一个无向图$G$，为找到它的最小割$(S,T)$， 随机选择一条边，将这条边无限变短直到两关联结点重合，然后继承除被缩短的边以外其余所有边，删去自环。

重复以上过程，直到图中只剩下两个结点。这两个结点代表的就是这样的一个最小割。

如下的无向图
```python
      1
    /  |
  /    |
2------3
|      |
|      |
4------5
```
合并结点1， 3后，得到
```python
 _______
/       \
2-------1, 3
|        |
|        |
4--------5
```
注意，此时（1， 2）,（3， 2）两条边虽然共同组成了一个环，但它们没有一个是自环，因此必须保留。接下来假设我们合并1， 2， 得到下图：
```python
________
\       |
 \      |
  \     |
  1, 2, 3
   /    |
 /      |
4-------5
```
注意此时1， 2， 3形成了自环，因此删去自环边，所以这一步骤结束后的图应当是
```python
  1, 2, 3
  /     |
 /       |
/        |
4--------5
```
最后合并1， 3， 4， 5， 得到一个只有两个结点的图，这两个结点就是最小割。



#### 2.3 Description of Karger's Algorithm

1. Pick an edge $e=(u,v)$ uniformly at random

2. Contract $e$ i.e. $u$ absorbs $v$
    - delete edges between $u$ and $v$
    - redirect edges from $v$ to $u$

3. Repeat until multi-graph has just two nodes $u_1$ and $v_1$ 

4. Return the cut $(S, S')$ where $S$ is the set of nodes absorbed by $u_1$

#### 2.4 Probability of Success

The contraction algorithms returns a min cut with
$$
\mathbb{P}(\text{success}) \geq \frac{2}{n(n-1)}
$$
where n is the number of vertices in the graph.

Let $C$ be a min cut with $k$ edges.

Let $e_{1},...,e_{n-2}$ be the contracted edges (output is C if none of them are in C)
- each vertex has degree at least $k \rightarrow$ there are at leat $\frac{nk}{2}$ edges
- $e_1$ is in $C$ with prob $\frac{k}{\#edges} \leq \frac{2}{n}$ and not in $C$ with prob $\geq \frac{n-2}{n}$
- if $e_1$ is not in $C$, then $C$ is still a mincut of contraction $(G, e_{1})$
- conditioned that $e_1, \ldots, e_{i-1}$ are not in $C, e_i$ is not in $C$ with prob $\geq \frac{n-i-1}{n-i+1}$

Therefore, the probability that $C$ is a min cut is 
$$
\frac{n-2}{n} \cdot \frac{n-3}{n-1} \cdot \frac{n-4}{n-2} ... \frac{2}{4} \cdot \frac{1}{3}= \frac{2}{n(n-1)}
$$
If we repeat the algorithm $n^2$ times, then the probability of failing to find the min cut is $\mathbb{P}(failing)=\frac{1}{n^2}$
$$
\left(1-\frac{2}{n^2}\right)^{n^2 \ln n}=\left[\left(1-\frac{2}{n^2}\right)^{\frac{1}{2} n^2}\right]^{2 \ln n} \leq\left(e^{-1}\right)^{2 \ln n}=\frac{1}{n^2}
$$
`Complexity:` $O(n^{4}\ln n)$

### 3. Quick Sort 快速排序

**Input:** an array of n comparable elements - a pivot p among them

**Output:**  array partitioned around p - new index of p. 

快速排序使用分治法来把一个串（list）分为两个子串（sub-lists）。具体算法描述如下：
- 从数列中挑出一个元素，称为 “基准”（pivot)
- 重新排序数列，所有元素比基准值小的摆放在基准前面，所有元素比基准值大的摆在基准的后面（相同的数可以到任一边）。在这个分区退出之后，该基准就处于数列的中间位置。这个称为分区（partition）操作
- 递归地（recursive）把小于基准值元素的子数列和大于基准值元素的子数列排序

[Quick Sort](https://blog.csdn.net/ytx2014214081/article/details/105841969)

In [6]:
def partition(A, lo, hi):
    p = A[lo]
    i = lo
    j = hi
    while True:
        for i in range(i + 1, hi):
            if A[i] >= p:
                break
        for j in range(j - 1, lo - 1, -1):
            if A[j] <= p:
                break
        if i >= j:
            break
        A[i], A[j] = A[j], A[i]
    A[lo], A[j] = A[j], A[lo]
    return j


def sort(A):
    quicksort(A, 0, len(A))


def quicksort(A, lo, hi):
    if hi <= lo + 1:
        return
    q = partition(A, lo, hi)
    quicksort(A, lo, q)
    quicksort(A, q + 1, hi)

A = [1, 2, 3, 1, 4, 2, 34, 154, 12, 3]
sort(A)
print(A)


def main():
    A = randMatrix(1, 1000).tolist()[0]
    B = A.copy()
    start = timeit.default_timer()
    sort(A)
    stop = timeit.default_timer()
    print('Time:', stop - start)
    start = timeit.default_timer()
    B.sort()
    stop = timeit.default_timer()
    print('Time:', stop - start)
    print(A == B)


[1, 1, 2, 2, 3, 3, 4, 12, 34, 154]


#### Complexity Analysis - Worst Case

Quadratic worst-case on a sorted array: 1 2 3 4 5 6

- 1 is compared with the $n-1$ other entries
- sorting is called recursively on $A[2:n]$

$(n-1)+(n-2)+...=\frac{n(n-1)}{2}=O(n^2)$ comparisons

#### Complexity Analysis - Average Case

**Strong Hypothesis:** the keys are distinct and all permutations of the input are equally likely

Therefore, we can use the expected number of comparisons as the running time and compute recursively to get the average case running time. $C_{n}$ := num. comparisons
$$
C_{0} = 0 
$$
$$
C_n=\underbrace{n-1}_{\text {partition }}+C_{i-1}+C_{n-i} \quad \text { if pivot at index } i
$$
Average number of comparisons $E_{n} := \mathbb{E}(C_n)$

The starting point is to decompose the expected number of comparisons according to the location of the pivot:
$$
\begin{aligned}
\mathbb{E}\left(C_n\right) &=\sum_{i=1}^n \mathbb{E}\left(C_n \mid \text { pivot at } i\right) \operatorname{Pr}(\text { pivot at } i) \\
&=\sum_{i=1}^n \mathbb{E}\left(n-1+C_{i-1}+C_{n-i}\right) \frac{1}{n},
\end{aligned}
$$
whence by linearity of expectation
$$
E_n=n-1+\sum_{i=1}^n \frac{E_{i-1}+E_{n-i}}{n} .
$$
By reorganization of the final sum, this becomes
$$
E_n=n-1+\frac{2}{n}\left(E_0+\cdots+E_{n-1}\right) .
$$
The sum will be disposed of by first isolating it
$$
n E_n-n(n-1)=2\left(E_0+\cdots+E_{n-1}\right)
$$
and subtracting two consecutive values, which leads to a simple linear recurrence
$$
n E_n-(n-1) E_{n-1}-2(n-1)=2 E_{n-1} \text {. }
$$
Rearranging and dividing by $n(n+1)$ gives
(2) $\frac{E_n}{n+1}-\frac{E_{n-1}}{n}=\frac{2(n-1)}{n(n+1)}=\frac{4}{n+1}-\frac{2}{n}$.
Now, the left-hand side telescopes when summing for $n=1$ to $N$ (we use that $\left.E_0=C_0=0\right)$, leading to
$$
\frac{E_N}{N+1}=4\left(\frac{1}{2}+\frac{1}{3}+\cdots+\frac{1}{N+1}\right)-2\left(1+\frac{1}{2}+\cdots+\frac{1}{N}\right)=2 H_N-\frac{4 N}{N+1},
$$
where
$$
H_N=1+\cdots+\frac{1}{N}=\log N+\gamma+O(1 / N), \quad N \rightarrow \infty
$$
is the Nth harmonic number, and $\gamma \approx 0.577$ denotes Euler's constant.



In [7]:
# randomize the input
import random
def sort(A):
    random.shuffle(A)
    quicksort(A,0,len(A))

Now, for arbitrarily bad input, the expected number of comparisons is $\simeq 2n\log n - 2.85n$